In [3]:
import h5py
import numpy as np
from ccfd_dnn.model import *
from ccfd_dnn.model_eval import *

Using Theano backend.
Using gpu device 0: GeForce GTX 980 (CNMeM is disabled, cuDNN 5103)
/usr/local/lib/python2.7/dist-packages/theano/sandbox/cuda/__init__.py:600: UserWarning:

Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.



In [4]:
table = 'data_little_enc'
w_id = 1

In [5]:
    disk_engine = get_engine()
    ml = ModelLoader(table,w_id)
    model = ml.model
    title = 'BiRNN-DO3-DLE'

Model LOADED!


In [6]:
model.__dict__

{'_function_kwargs': {},
 '_output_mask_cache': {'140177944842064_140177756121744': All{2}.0,
  '140177944842064_9545840': All{2}.0},
 '_output_shape_cache': {'(None, 60, 44)': (None, 60, 3)},
 '_output_tensor_cache': {'140177944842064_9545840': Reshape{3}.0},
 'built': True,
 'container_nodes': {'gru_1_ib-0',
  'gru_2_ib-0',
  'gru_3_ib-0',
  'gru_4_ib-0',
  'gru_5_ib-0',
  'gru_6_ib-0',
  'gru_7_ib-0',
  'gru_8_ib-0',
  'main_input_ib-0',
  'masking_1_ib-0',
  'merge_1_ib-0',
  'merge_2_ib-0',
  'merge_3_ib-0',
  'merge_4_ib-0',
  'timedistributed_1_ib-0'},
 'inbound_nodes': [<keras.engine.topology.Node at 0x7f7dad5d20d0>],
 'input_layers': [<keras.engine.topology.InputLayer at 0x7f7db89b8f90>],
 'input_layers_node_indices': [0],
 'input_layers_tensor_indices': [0],
 'input_names': ['main_input'],
 'inputs': [main_input],
 'internal_input_shapes': [(None, 60, 44)],
 'internal_output_shapes': [(None, 60, 3)],
 'layers': [<keras.engine.topology.InputLayer at 0x7f7db89b8f90>,
 'layers_b

In [16]:
model.layers[0].inbound_nodes[0].__dict__

{'inbound_layers': [],
 'input_masks': [None],
 'input_shapes': [(None, 60, 44)],
 'input_tensors': [main_input],
 'node_indices': [],
 'outbound_layer': <keras.engine.topology.InputLayer at 0x7f7db89b8f90>,
 'output_masks': [None],
 'output_shapes': [(None, 60, 44)],
 'output_tensors': [main_input],
 'tensor_indices': []}

In [27]:
model.layers[2].__dict__

{'U': gru_1_U,
 'U_regularizer': None,
 'W': gru_1_W,
 'W_regularizer': None,
 'activation': <function keras.activations.tanh>,
 'b': gru_1_b,
 'b_regularizer': None,
 'batch_input_shape': (None, None, 44),
 'built': True,
 'constraints': {},
 'consume_less': 'gpu',
 'dropout_U': 0.0,
 'dropout_W': 0.3,
 'go_backwards': False,
 'inbound_nodes': [<keras.engine.topology.Node at 0x7f7db836ded0>],
 'init': <function keras.initializations.glorot_uniform>,
 'initial_weights': None,
 'inner_activation': <function keras.activations.hard_sigmoid>,
 'inner_init': <function keras.initializations.orthogonal>,
 'input_dim': 44,
 'input_dtype': 'float32',
 'input_length': None,
 'input_spec': [<keras.engine.topology.InputSpec at 0x7f7db891ce10>],
 'name': 'gru_1',
 'non_trainable_weights': [],
 'outbound_nodes': [<keras.engine.topology.Node at 0x7f7db82ae110>],
 'output_dim': 320,
 'regularizers': [],
 'return_sequences': True,
 'stateful': False,
 'states': [None],
 'supports_masking': True,
 'trai

In [30]:
inter_models = []
for layer in model.layers:
    if type(layer) == keras.engine.topology.Merge:
        print layer.__dict__
        inter_models.append(Model(input=model.inputs, output=layer.output))

{'layers': None, 'regularizers': [], 'outbound_nodes': [<keras.engine.topology.Node object at 0x7f7db80c2d90>, <keras.engine.topology.Node object at 0x7f7db03f4dd0>], 'trainable_weights': [], 'name': 'merge_1', 'concat_axis': -1, 'supports_masking': True, 'uses_learning_phase': False, 'non_trainable_weights': [], 'dot_axes': [-1, -1], '_output_shape': None, 'mode': 'concat', 'built': True, '_output_mask': None, 'inbound_nodes': [<keras.engine.topology.Node object at 0x7f7db82ae110>], 'input_spec': None, 'node_indices': None, 'constraints': {}}
{'layers': None, 'regularizers': [], 'outbound_nodes': [<keras.engine.topology.Node object at 0x7f7db02dd7d0>, <keras.engine.topology.Node object at 0x7f7db00bb990>], 'trainable_weights': [], 'name': 'merge_2', 'concat_axis': -1, 'supports_masking': True, 'uses_learning_phase': False, 'non_trainable_weights': [], 'dot_axes': [-1, -1], '_output_shape': None, 'mode': 'concat', 'built': True, '_output_mask': None, 'inbound_nodes': [<keras.engine.top

In [28]:
inter_models = []
for layer in model.merge_1_ib-0:
    if type(layer) == keras.engine.topology.Merge:
        print layer.__dict__
        print type(layer())
        inter_models.append(Model(input=model.inputs, output=layer.output))

AttributeError: 'Model' object has no attribute 'merge_1_ib'

In [18]:
test_layer = Input(shape=(None,60,44),name='test')
type(test_layer)

theano.tensor.var.TensorVariable

In [20]:
mask = Masking(mask_value=0)(test_layer)
type(mask)

Exception: Input 0 is incompatible with layer dense_1: expected ndim=2, found ndim=4

In [ ]:
input_layer = Input(shape=(int(seq_len_param), 44),name='main_input')
mask = Masking(mask_value=0)(input_layer)
